In [ ]:
    def generate_ect_prompts(self, ect: str, id_bb_company: str, html_prompt: str = "") -> List[Dict]:
        """Generate ECT analysis prompts"""
        items = list(ECTConfig.HEADINGS.items())
        bucket_size = 4
        prompt_buckets = [items[i:i + bucket_size] for i in range(0, len(items), bucket_size)]
        
        prompts = []
        for bucket_id, bucket in enumerate(prompt_buckets):
            prompt_str = self._format_bucket_prompt(bucket, ect, html_prompt)
            prompts.append({
                "prompt": prompt_str,
                "company": id_bb_company,
                "type": "ECT",
                "bucket": bucket_id,
                "model": "gpt-4o"
            })
        return prompts

    def _format_bucket_prompt(self, bucket: List[tuple], ect: str, html_prompt: str) -> str:
        """Format the prompt for each bucket"""
        metric_count = len(bucket)
        prompt = f" Answer the following {metric_count} questions using content from the Earnings call text specified below. "
        prompt += "Make sure to include all supporting information available in the text without repeating information.\n"
        prompt += f"Make sure the response is able to answer the {metric_count} questions below {html_prompt}\n"
        
        for i, (heading, metrics) in enumerate(bucket, 1):
            metrics_str = ", ".join(metrics)
            prompt += f"{i}. How is the {metrics_str} of the company? "
            prompt += f"Give an in-depth response under the sub-heading: {heading}. "
            prompt += "Use up to 7 bullets to formulate the response.\n"
            
        prompt += f"The Earnings call text is as follows: \n{ect}"
        return prompt